## Decrypting Your Grades

To protect your privacy, I have encrypted your grades in the CSV file "grades_encrypted.csv". This means that your grades are securely stored and can only be accessed by you using your unique ID number. To decrypt and view your grades, please follow the instructions below.

1. Download the encrypted CSV file containing your grades from the github repository.
2. Open this Jupyter Notebook.
3. Update the csv_file_path variable in the code with the path to the downloaded encrypted CSV file.
4. Run the code cell.
5. When prompted, enter your ID number and press Enter.
6. The code will decrypt your grades and display them if found.

In [ ]:
%pip install pandas cryptography

In [ ]:
import pandas as pd
from cryptography.fernet import Fernet
import base64
import hashlib

def derive_key(id_number):
    id_bytes = str(id_number).encode()
    key = hashlib.sha256(id_bytes).digest()
    encoded_key = base64.urlsafe_b64encode(key)
    return encoded_key

def decrypt_value(encrypted_value, key):
    try:
        fernet = Fernet(key)
        decrypted_value = fernet.decrypt(encrypted_value.encode()).decode()
        return decrypted_value
    except Exception as e:
        return None

def decrypt_grades(id_number, csv_file_path):
    try:
        df = pd.read_csv(csv_file_path)
        df['Decrypted_id_number'] = df['Encrypted_id_number'].apply(lambda x: decrypt_value(x, derive_key(id_number)))
        row = df.loc[df['Decrypted_id_number'] == id_number]
        if len(row) == 0:
            print(f"No grades found for ID number: {id_number}")
            return None
        key = derive_key(id_number)
        decrypted_row = {}
        for column in row.columns:
            if column.startswith('Encrypted_'):
                encrypted_value = row[column].values[0]
                if pd.notnull(encrypted_value):
                    decrypted_value = decrypt_value(encrypted_value, key)
                    decrypted_row[column[10:]] = decrypted_value
                else:
                    decrypted_row[column[10:]] = None

        return decrypted_row
    except Exception as e:
        print(f"Error decrypting grades: {str(e)}")
        return None

id_number = input("Enter your ID number: ")

csv_file_path = "grades_encrypted.csv"

decrypted_grades = decrypt_grades(id_number, csv_file_path)

if decrypted_grades is not None:
    print("Decrypted Grades:")
    for column, value in decrypted_grades.items():
        print(f"{column}: {value}")